In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS
from statsmodels.tools import add_constant

In [45]:
def regressHart(y, X, Z, **kwargs):
    n = len(y)
    if n != X.shape[0]:
        raise ValueError('Size mismatch')
    maxiterdef = 10000
    toldef = 1e-08
    test=0
    options = dict(
        intercept=True,
        maxiter=maxiterdef,
        type='art',
        initialbeta=None, # shall be array of shape (n_features, ) for single target regression
        initialgamma=None,
        tol=toldef,
        nocheck=False,
        msgiter=0,
        test=test
    )
    test = options['test']
    reg = LinearRegression(fit_intercept=options['intercept'])
    # number of predictors
    if options['intercept']:
        p = X.shape[1] + 1
    else:
        p = X.shape[1]
    reg_results0 = reg.fit(X, y)
    b0 = reg_results0.coef_ # b0 is array of shape (n_features, ) for single target regression
    b0 = np.c_[reg_results0.intercept_, b0]
    r = y - reg_results0.predict(X)
    oldbeta = b0
    sigma2 = r.T @ r / (n-p)
    print(sigma2)
    # loglikelihood of residual
    logL_R = n * (1 + np.log(sigma2))
    #  Initialization of gamma
    #  Z = n-by-r matrix which contains the explanatory variables for
    #  heteroskedasticity
    Z_ = Z.copy
    if options['intercept']:
        Z_ = add_constant(Z_)
    response_gamma0 = np.square(r)
    reg_gamma = reg.fit(Z, response_gamma0)
    gamma0 = reg_gamma.coef_
    gamma0 = np.c_[reg_gamma.intercept_, gamma0]
    oldgamma = gamma0
    # tolerance
    tol = options['tol']

    



In [47]:
r = np.array([2, 3, 4])
np.square(r)

array([ 4,  9, 16], dtype=int32)

In [20]:
trade_df = pd.read_csv('TradeH.csv', sep='\t', header=None, names=['quantity', 'value'])

X = trade_df['quantity'].values.reshape(-1, 1)
y = trade_df['value'].values

In [37]:
lm = OLS(y, add_constant(X))
results = lm.fit()
sigma2 = results.get_prediction(add_constant(X)).var_resid
sigma2

6159773.924592378

In [36]:
results.params

array([1011.81221248,    9.3880973 ])

In [46]:
regressHart(y, X)

6159773.924592377
